<a href="https://colab.research.google.com/github/Gialbo/COVID-Chest-X-Rays-Deep-Learning-analysis/blob/main/experiments/compute_FID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and data

In [ ]:
# IMPORT AND MOUNT DRIVE
import numpy as np
from numpy.random import randint
from keras.applications.inception_v3 import InceptionV3
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import os.path
import sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# CLONE GITHUB REPOSITORY
!rm -r "COVID-Chest-X-Rays-Deep-Learning-analysis"
!git clone https://github.com/Gialbo/COVID-Chest-X-Rays-Deep-Learning-analysis.git

rm: cannot remove 'COVID-Chest-X-Rays-Deep-Learning-analysis': No such file or directory
Cloning into 'COVID-Chest-X-Rays-Deep-Learning-analysis'...
remote: Enumerating objects: 1293, done.
remote: Counting objects: 100% (704/704), done.
remote: Compressing objects: 100% (537/537), done.
remote: Total 1293 (delta 434), reused 384 (delta 167), pack-reused 589
Receiving objects: 100% (1293/1293), 110.36 MiB | 26.14 MiB/s, done.
Resolving deltas: 100% (697/697), done.


In [ ]:
# LOAD DIRECTORIES
sys.path.append('COVID-Chest-X-Rays-Deep-Learning-analysis/models')
sys.path.append('COVID-Chest-X-Rays-Deep-Learning-analysis/tools')

In [ ]:
# LOAD VALIDATION DATA

from XRaysDataset import XRaysDataset

dataset = XRaysDataset(batch_size=383, dir='/content/drive/MyDrive/BIOINF/covid-project/dataset/test')
test_ds, test_size = dataset.load(separate_classes=False)

test_covid, test_covid_size = dataset.load(covid_class=True)

Dataset size  383
Number of batches for the dataset: 1
Number of batches for the covid dataset: 1


In [ ]:
# Obtain the batch of real images
for real_images, _ in test_ds:
  pass

print("Validation set shape: ", real_images.shape)


for real_images_covid in test_covid:
  pass

print("Validation set covid shape: ", real_images_covid.shape)

batch_size = 383
batch_size_covid = 115

Validation set shape:  (383, 128, 128, 1)
Validation set covid shape:  (115, 128, 128, 1)


# Load the Inception model and FID class

In [ ]:
# prepare the inception v3 model
from FID import FID
evaluation_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299,3))

# FID_covid is used when the generative model is able to generate sample from just one class (covid-19)
FID_covid = FID(batch_size=batch_size_covid)

# FID is used when the generative model is able to generate samples from all the three classes
FID = FID(batch_size=batch_size)

87924736/87910968 [==============================] - 3s 0us/step


# Compute FID

## Covid-GAN

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE
from covidGAN import covidGAN
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/covidGAN' #

generative_model = covidGAN()
checkpoint = tf.train.Checkpoint(gan_optimizer=generative_model.gan_optimizer,
                                    gan=generative_model.gan)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

covidGAN created
/content/drive/MyDrive/BIOINF/checkpoints_GAN/covidGAN/ckpt-9


In [ ]:
mean_FID_covidGAN, std_FID_covidGAN = FID_covid.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images_covid, n_classes=1)
print("------- covidGAN model -------")
print("Mean FID: ", mean_FID_covidGAN)
print("STD FID: ", std_FID_covidGAN)

Trial  0
FID:  309.9657387488619
Trial  1
FID:  316.2103952979721
Trial  2
FID:  316.27179276428393
Trial  3
FID:  314.7765622354287
Trial  4
FID:  312.0194327125697
------- covidGAN model -------
Mean FID:  313.84878435182327
STD FID:  2.4792546623471305


## Covid-Unet-GAN

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE
from covidUnetGAN import covidUnetGAN
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/covidUnetGAN' #

generative_model = covidUnetGAN()
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.model.generator_optimizer,
                                  discriminator_optimizer=generative_model.model.discriminator_optimizer,
                                  model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

covidUnetGAN created
/content/drive/MyDrive/BIOINF/checkpoints_GAN/covidUnetGAN/ckpt-9


In [ ]:
mean_FID_covidUnetGAN, std_FID_covidUnetGAN = FID_covid.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images_covid, n_classes=1)
print("------- covidUnetGAN model -------")
print("Mean FID: ", mean_FID_covidUnetGAN)
print("STD FID: ", std_FID_covidUnetGAN)

Trial  0
FID:  193.6458308589422
Trial  1
FID:  185.35544561407235
Trial  2
FID:  186.36920222259715
Trial  3
FID:  184.45509296698285
Trial  4
FID:  192.57604598782734
------- covidUnetGAN model -------
Mean FID:  188.48032353008438
STD FID:  3.8439975199361602


## cGAN

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE

from cGAN import cGAN
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/cGAN' #

generative_model = cGAN(use_residual=True)
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.generator_optimizer,
                                    discriminator_optimizer=generative_model.discriminator_optimizer,
                                    model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

/content/drive/MyDrive/BIOINF/checkpoints_GAN/cGAN/ckpt-11


In [ ]:
mean_FID_cGAN, std_FID_cGAN = FID.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images)
print("------- cGAN model -------")
print("Mean FID: ", mean_FID_cGAN)
print("STD FID: ", std_FID_cGAN)

Trial  0
FID:  80.93746087947187
Trial  1
FID:  82.6431745070553
Trial  2
FID:  78.8950117959555
Trial  3
FID:  80.99389993192827
Trial  4
FID:  79.77940116708977
------- cGAN model -------
Mean FID:  80.64978965630014
STD FID:  1.2658255202799327


## cGAN - Max Uncertainty

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE

from cGAN_Uncertainty import cGANUnc
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/cGAN-unc-max' #

generative_model = cGANUnc(use_residual=True)
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.generator_optimizer,
                                    discriminator_optimizer=generative_model.discriminator_optimizer,
                                    model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

/content/drive/MyDrive/BIOINF/checkpoints_GAN/cGAN-unc-max_OLD/ckpt-6


In [ ]:
mean_FID_cGAN, std_FID_cGAN = FID.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images)
print("------- cGAN model -------")
print("Mean FID: ", mean_FID_cGAN)
print("STD FID: ", std_FID_cGAN)

Trial  0
FID:  62.25240717632714
Trial  1
FID:  63.70093656933638
Trial  2
FID:  62.370191615359865
Trial  3
FID:  61.934819735966485
Trial  4
FID:  62.67218057109943
------- cGAN model -------
Mean FID:  62.58610713361786
STD FID:  0.6053765364208147


## cGAN - Min Uncertainty

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE

from cGAN_Uncertainty import cGANUnc
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/cGAN_unc_min' #

generative_model = cGANUnc(use_residual=True)
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.generator_optimizer,
                                    discriminator_optimizer=generative_model.discriminator_optimizer,
                                    model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

/content/drive/MyDrive/BIOINF/checkpoints_GAN/cGAN_unc_min/ckpt-11


In [ ]:
mean_FID_cGAN, std_FID_cGAN = FID.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images)
print("------- cGAN model -------")
print("Mean FID: ", mean_FID_cGAN)
print("STD FID: ", std_FID_cGAN)

Trial  0
FID:  73.23192046742025
Trial  1
FID:  71.64625870048712
Trial  2
FID:  71.62457861754154
Trial  3
FID:  73.96154456096198
Trial  4
FID:  72.92351735707216
------- cGAN model -------
Mean FID:  72.67756394069661
STD FID:  0.9152900157030532


## Unet-cGAN

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE

from unetCGAN import unetCGAN
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/unetCGAN' #

generative_model = unetCGAN(use_residual=True)
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.generator_optimizer,
                                  discriminator_optimizer=generative_model.discriminator_optimizer,
                                  model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

/content/drive/MyDrive/BIOINF/checkpoints_GAN/unetCGAN/ckpt-6


In [ ]:
mean_FID_unetCGAN, std_FID_unetCGAN = FID.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images)
print("------- unetCGAN model -------")
print("Mean FID: ", mean_FID_unetCGAN)
print("STD FID: ", std_FID_unetCGAN)

Trial  0
FID:  86.18821451886397
Trial  1
FID:  90.79460855279302
Trial  2
FID:  92.17353351188913
Trial  3
FID:  90.52669365789339
Trial  4
FID:  89.1346439453599
------- unetCGAN model -------
Mean FID:  89.76353883735987
STD FID:  2.0313560562864486


## AC-cGAN

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE

from ACCGAN import ACCGAN
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/AC-cGAN' #

generative_model = ACCGAN(use_residual=True)
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.generator_optimizer,
                                    discriminator_optimizer=generative_model.discriminator_optimizer,
                                    model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

/content/drive/MyDrive/BIOINF/checkpoints_GAN/AC-cGAN/ckpt-9


In [ ]:
mean_FID_ACcGAN, std_FID_ACcGAN = FID.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images)
print("------- AC-cGAN model -------")
print("Mean FID: ", mean_FID_ACcGAN)
print("STD FID: ", std_FID_ACcGAN)

Trial  0
FID:  83.86847869529885
Trial  1
FID:  81.15680585908868
Trial  2
FID:  84.24178235976059
Trial  3
FID:  79.62840186942518
Trial  4
FID:  80.4475903731681
------- AC-cGAN model -------
Mean FID:  81.86861183134828
STD FID:  1.853424734414901


## AC-cGAN - Min Uncertainty

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE

from ACCGAN_Uncertainty import ACCGANUnc
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/AC-CGAN_min_unc' #

generative_model = ACCGANUnc(use_residual=True)
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.generator_optimizer,
                                    discriminator_optimizer=generative_model.discriminator_optimizer,
                                    model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

/content/drive/MyDrive/BIOINF/checkpoints_GAN/AC-CGAN-uncertainty/gilberto/mcd_4_unc_weight_1_disc_min/ckpt-9


In [ ]:
mean_FID_ACcGAN, std_FID_ACcGAN = FID.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images)
print("------- AC-cGAN model -------")
print("Mean FID: ", mean_FID_ACcGAN)
print("STD FID: ", std_FID_ACcGAN)

Trial  0
FID:  87.82770999028202
Trial  1
FID:  91.36861491611162
Trial  2
FID:  90.29781190781272
Trial  3
FID:  90.75496794068766
Trial  4
FID:  88.02375760181147
------- AC-cGAN model -------
Mean FID:  89.65457247134108
STD FID:  1.4532413651865255


## AC-cGAN - Max Uncertainty

In [ ]:
# LOAD MODEL AND CHECKPOINT FROM DRIVE

from ACCGAN_Uncertainty import ACCGANUnc
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/AC-cGAN_max_unc' #

generative_model = ACCGANUnc(use_residual=True)
checkpoint = tf.train.Checkpoint(generator_optimizer=generative_model.generator_optimizer,
                                    discriminator_optimizer=generative_model.discriminator_optimizer,
                                    model=generative_model.model)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
checkpoint.restore(latest)

/content/drive/MyDrive/BIOINF/checkpoints_GAN/AC-cGAN_max_unc/ckpt-11


In [ ]:
mean_FID_ACcGAN, std_FID_ACcGAN = FID.get_FID(generative_model=generative_model, evaluation_model=evaluation_model, real_images=real_images)
print("------- AC-cGAN model -------")
print("Mean FID: ", mean_FID_ACcGAN)
print("STD FID: ", std_FID_ACcGAN)

Trial  0
FID:  77.25859242600411
Trial  1
FID:  77.34854883271191
Trial  2
FID:  77.30256556610719
Trial  3
FID:  79.64163360317278
Trial  4
FID:  81.16847439116702
------- AC-cGAN model -------
Mean FID:  78.54396296383261
STD FID:  1.5946912508795166
